In [ ]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

/home/ubuntu/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-06-29 21:41:29.611293: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 21:41:29.719320: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-29 21:41:30.274679: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.c

In [ ]:
from packages.TIMITDataLoader import TIMITDataLoader
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager
from packages.CadenceModelManager import CadenceModelManager
import packages.AnalysisManager as am
from packages.SmileFeatureManager import SmileFeatureManager

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
timit_data_loader = TIMITDataLoader('/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs')
#df = timit_data_loader.generate_split_speaker_test()
df = timit_data_loader.generate_split()
df['speaker'] = [item.split('/')[-1].split('_')[0] for item in df['path']]


fake_voices = ['Adam', 'Antoni', 'Arnold', 'Bella', 'Biden', 'Domi', 'Elli', 'Josh', 'Obama', 'Rachel', 'Sam']

real_speakers = list(set([item for item in df['speaker'] if not item.startswith(tuple(fake_voices))]))
fake_speakers = list(set([item for item in df['speaker'] if item.startswith(tuple(fake_voices))]))

real_speaker_partitions = list(chunks(real_speakers, 20))
fake_speaker_partitions = list(chunks(fake_speakers, 2))


N real and fake phrases: 500, 500
492 491
/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SI946/real/MPAR0_SI946.WAV
# of Train instances: 589
# of Dev instances: 196
# of Test instances: 198


In [ ]:
len(real_speakers + fake_speakers)

352

In [ ]:
df['path'][0]

'/home/ubuntu/data/TIMIT_and_ElevenLabs/16KHz/MPAR0_SI946.WAV'

In [ ]:
#df

In [ ]:
'''
import os
import shutil

file_paths = list(df['orig_path'])
output_dir = '/home/ubuntu/data/TIMIT_and_ElevenLabs/Original'

def copy_files_to_directory(file_paths, destination_directory):
    for file_path in file_paths:
        # Extract the file name from the path
        file_name = os.path.basename(file_path)
        
        # Create the destination path by joining the destination directory and the file name
        destination_path = os.path.join(destination_directory, file_name)
        
        try:
            # Copy the file to the destination directory
            shutil.copy2(file_path, destination_path)
            print(f"Successfully copied {file_path} to {destination_path}")
        except FileNotFoundError:
            print(f"File {file_path} not found.")
        except shutil.Error as e:
            print(f"Error occurred while copying {file_path}: {e}")
            
copy_files_to_directory(file_paths, output_dir)
'''

'\nimport os\nimport shutil\n\nfile_paths = list(df[\'orig_path\'])\noutput_dir = \'/home/ubuntu/data/TIMIT_and_ElevenLabs/Original\'\n\ndef copy_files_to_directory(file_paths, destination_directory):\n    for file_path in file_paths:\n        # Extract the file name from the path\n        file_name = os.path.basename(file_path)\n        \n        # Create the destination path by joining the destination directory and the file name\n        destination_path = os.path.join(destination_directory, file_name)\n        \n        try:\n            # Copy the file to the destination directory\n            shutil.copy2(file_path, destination_path)\n            print(f"Successfully copied {file_path} to {destination_path}")\n        except FileNotFoundError:\n            print(f"File {file_path} not found.")\n        except shutil.Error as e:\n            print(f"Error occurred while copying {file_path}: {e}")\n            \ncopy_files_to_directory(file_paths, output_dir)\n'

In [ ]:
df['orig_path'][977]

'/home/ubuntu/data/TIMIT_and_ElevenLabs/TIMIT and ElevenLabs/SX176/fake/Antoni_SX176.wav'

In [ ]:
#df[df['label'] == 1]

In [ ]:
'''
import os, shutil
paths = df['path'].tolist()

for path in paths:
    file_name = os.path.basename(path)
    output_path = f'/home/ubuntu/data/TIMIT_and_ElevenLabs/Original/{file_name}'
    shutil.copyfile(path, output_path)
'''

"\nimport os, shutil\npaths = df['path'].tolist()\n\nfor path in paths:\n    file_name = os.path.basename(path)\n    output_path = f'/home/ubuntu/data/TIMIT_and_ElevenLabs/Original/{file_name}'\n    shutil.copyfile(path, output_path)\n"

In [ ]:
### End of timit additions ###

In [ ]:
def generate_features(data_df):
    speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')
    embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)
    em_feature_df, em_feature_cols = embedding_manager.generateFeatureDf()
    
    cadence_manager = CadenceModelManager(data_df)
    cad_feature_df, cad_feature_cols, scalar =  cadence_manager.run_cadence_feature_extraction_pipeline(fill_na=-1) # Add param for load features or not
    
    smile_manager = SmileFeatureManager(data_df)
    #change number of features (feature_count=10 default)
    os_binary_feature_df, os_binary_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='binary', feature_count=20)
    os_multiclass_feature_df, os_multiclass_feature_cols = smile_manager.generateFeatureDf('random_forest', label_type='multiclass', feature_count=20)
    
    feature_store = {}
    feature_store['titanet'] = (em_feature_df, em_feature_cols)
    feature_store['openSmile_binary'] = (os_binary_feature_df, os_binary_feature_cols)
    feature_store['openSmile_multiclass'] = (os_multiclass_feature_df, os_multiclass_feature_cols)
    feature_store['cadence'] = (cad_feature_df, cad_feature_cols)
    
    return feature_store
    
    

In [ ]:
def train_eval(feature_store, fake_cols):
    results_cols = ['feature_method', 'model', 'fake_cols', 'label_type', 'acc', 'cls_acc', 'loss', 'eer_score', 'eer_threshold']
    results = pd.DataFrame(columns=results_cols)
    
    models = ['logreg', 'random_forest']
    for model in models:
        for label_type in ['label', 'multiclass_label']:
            for k, v in feature_store.items():
                model_manager = ModelManager(model, v[0], v[1], merge_train_dev=True)
                model_manager.trainPredict(label_col=label_type)
                results = results.append(pd.DataFrame({'feature_method':[k], 'label_type':[label_type], 'fake_cols':[fake_cols], 'acc':[model_manager.accuracy], 'cls_acc':[model_manager.class_accuracy],  'loss':[model_manager.log_loss_value], 'model':[model], 
                                                       'eer_score':[model_manager.eer_score], 'eer_threshold':[model_manager.eer_threshold]}))
    
    return results
    
    

In [ ]:
def run(fake_cols, metadata_path, name, loader, data_df=None, csv=True, speakers_to_remove=real_speaker_partitions[2]+fake_speaker_partitions[3]):
    if data_df is None:
        data_df = loader.generate_split_speaker_test(speakers_to_remove)
        data_df.to_csv(f'/home/ubuntu/data/results/TIMIT/DATA_DF_{name}.csv')
    feature_store = generate_features(data_df)
    results = train_eval(feature_store, fake_cols)
    
    if csv:
        results.to_csv(f'/home/ubuntu/data/results/TIMIT/{name}.csv', index=False)
    
    return results
    

In [ ]:
from IPython.display import clear_output

file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
# Single results run
#run(['ThisDoesNotMatter'], file_path, '16KHz_ElevenLabs_TIMIT_single_fold_TEST', loader=timit_data_loader)

# Multiple results run
results = []
counter = 0 

for fake_speaker_chunk in fake_speaker_partitions:
    for real_speaker_chunk in real_speaker_partitions:
        print(f'progress: {counter}/{len(fake_speaker_partitions)*len(real_speaker_partitions)}')
        voices_to_remove = fake_speaker_chunk+real_speaker_chunk
        results.append(run(['ThisDoesNotMatter'], file_path, '16KHz_ElevenLabs_TIMIT_k_fold_final', loader=timit_data_loader, csv=False))
        counter += 1
        clear_output()


In [ ]:
results_df = pd.DataFrame(results[0])

for i in range(len(results[1:])):
    results_df = pd.concat([results_df, results[i]])

results_df = results_df.reset_index()

results_df['class_acc_0'] = results_df['cls_acc'].apply(pd.Series)[0]
results_df['class_acc_1'] = results_df['cls_acc'].apply(pd.Series)[1]

results_df.groupby(['feature_method', 'model', 'label_type'])[['acc', 'class_acc_0', 'class_acc_1', 'eer_score']].mean().reset_index().to_csv('/home/ubuntu/data/results/TIMIT/16KHz_ElevenLabs_TIMIT_k_fold_regenerated.csv')

# RUN DOWN TO HERE! ^ 

For the columns, we want:
- Which dataset are we using
- What are the fake cols
- Binary/Multi-Class
- Feature Generation Method
- Accuracy
- Loss